In [5]:
import pandas as pd
import numpy as np

# ==================== CONFIGURATION ====================
DATE_START = pd.to_datetime('2022-01-01', utc=True)
DATE_END = pd.to_datetime('2025-12-31', utc=True)
EXCLUDE_COLS_HOURLY = ["numero_poste","nom_usuel","latitude","longitude","altitude","duree_precipitations", "vent_moyen", "code_etat_neige", "charge_neige",
                       "neige_au_sol", "code_etat_sol_sans_neige", "code_etat_sol_avec_neige"]
EXCLUDE_COLS_ALERT = ["type_vigilance"]

def load_and_filter_data(filepath, date_col_pattern='date', date_start=DATE_START, date_end=DATE_END, exclude_cols=None, dept_number=92):
    """Load CSV and apply standard filters (date range, department, column exclusion)."""
    df = pd.read_csv(filepath)
    
    # Auto-detect datetime column
    dt_col = next((col for col in df.columns if date_col_pattern in col.lower() or 'time' in col.lower()), df.columns[0])
    
    # Convert all date columns to datetime
    for col in df.columns:
        if 'date' in col.lower():
            df[col] = pd.to_datetime(df[col])
    
    # Normalize timezone
    if df[dt_col].dt.tz is not None:
        df[dt_col] = df[dt_col].dt.tz_convert('UTC')
    else:
        df[dt_col] = df[dt_col].dt.tz_localize('UTC')
    
    # Filter by date range
    df = df[(df[dt_col] >= date_start) & (df[dt_col] <= date_end)]
    
    # Filter by department (keep only department 92)
    dept_cols = [col for col in df.columns if 'department' in col.lower() or 'num_departement' in col.lower() or 'departement' in col.lower()]
    if dept_cols:
        dept_col = dept_cols[0]
        df = df[df[dept_col] == dept_number]
    
    # Exclude columns
    if exclude_cols:
        df = df[[col for col in df.columns if col not in exclude_cols]]
    
    return df, dt_col

# ==================== LOAD & FILTER DATA ====================
print("Loading hourly weather data...")
df_hourly_filtered, datetime_col = load_and_filter_data(
    r"C:\Users\marcs\Documents\GitHub\IDFm-Hackathon-Team9\raw-data\meteo_france_horaire_2020-2025\meteo_france_horaire_2020-2025.csv",
    exclude_cols=EXCLUDE_COLS_HOURLY
)
print(f"  ✓ Hourly weather: {len(df_hourly_filtered)} rows, {len(df_hourly_filtered.columns)} cols")

print("Loading alert data...")
df_alerts_filtered, datetime_col_alert = load_and_filter_data(
    r"C:\Users\marcs\Documents\GitHub\IDFm-Hackathon-Team9\raw-data\03_Vigilance_meteo_20221127_20250722\vigilance_data_from_2022-11-27_to_2025-07-22.csv",
    exclude_cols=EXCLUDE_COLS_ALERT
)
print(f"  ✓ Alerts: {len(df_alerts_filtered)} rows, {len(df_alerts_filtered.columns)} cols")

# Convert datetime columns to string format for matching (project convention)
df_hourly_filtered[datetime_col] = df_hourly_filtered[datetime_col].dt.strftime('%Y-%m-%d %H:%M:%S')
for col in df_alerts_filtered.columns:
    if 'date' in col.lower():
        df_alerts_filtered[col] = df_alerts_filtered[col].dt.strftime('%Y-%m-%d %H:%M:%S')

# ==================== CREATE PIVOT TABLE: DATETIME × PHENOMENA ====================

def create_phenomena_vigilance_pivot(df_hourly, df_alerts, datetime_col_hourly='datetime',
                                      datetime_col_alert='datetime_debut_vigilance', 
                                      phenomene_col='phenomene_id', vigilance_col='niveau_vigilance',
                                      date_start_col='date_debut_vigilance', date_end_col='date_fin_vigilance',
                                      quantite_precipitations='quantite_precipitations',
                                      temperature_instant='temperature_instant',
                                      num_phenomena=10, default_vigilance='Vert'):
    """
    Create a pivot table with datetime rows, weather columns, and phenomena (1-10) columns.
    Each cell contains the niveau_vigilance for that datetime and phenomenon, plus weather data.
    
    Parameters:
    -----------
    df_hourly : DataFrame
        Hourly weather data with datetime column and weather columns
    df_alerts : DataFrame
        Alert data with phenomenon, vigilance level, and date range columns
    datetime_col_hourly : str
        Name of datetime column in df_hourly
    datetime_col_alert : str
        Name of datetime alert column (unused, kept for compatibility)
    phenomene_col : str
        Name of phenomenon ID column in df_alerts
    vigilance_col : str
        Name of vigilance level column in df_alerts
    date_start_col : str
        Name of alert start datetime column in df_alerts
    date_end_col : str
        Name of alert end datetime column in df_alerts
    quantite_precipitations : str
        Name of precipitation quantity column in df_hourly
    temperature_instant : str
        Name of instantaneous temperature column in df_hourly
    num_phenomena : int
        Number of phenomena columns to create (1 to num_phenomena)
    default_vigilance : str
        Default value when no alert matches (e.g., 'Vert', 'Green', None)
    
    Returns:
    --------
    DataFrame
        Pivot table with columns: ['datetime', 'quantite_precipitations', 'temperature_instant', 'phenomene_1', ..., 'phenomene_N']
    """
    
    # Weather columns to include from df_hourly
    weather_cols = [quantite_precipitations, temperature_instant]
    
    # Get unique datetimes from hourly data (as strings)
    unique_datetimes = sorted(df_hourly[datetime_col_hourly].unique())
    
    # Initialize result dataframe
    pivot_data = []
    
    # For each unique datetime
    for dt_str in unique_datetimes:
        row = {'datetime': dt_str}
        
        # Convert to datetime for comparison
        dt = pd.to_datetime(dt_str)
        
        # Add weather columns
        for weather_col in weather_cols:
            # Get the weather value for this datetime
            weather_values = df_hourly[df_hourly[datetime_col_hourly] == dt_str][weather_col]
            if len(weather_values) > 0:
                row[weather_col] = weather_values.iloc[0]
            else:
                row[weather_col] = None
        
        # Convert alert dates to datetime for comparison
        alert_start = pd.to_datetime(df_alerts[date_start_col])
        alert_end = pd.to_datetime(df_alerts[date_end_col])
        
        # For each phenomenon (1 to num_phenomena)
        for phenom_id in range(1, num_phenomena + 1):
            # Find matching alert for this datetime and phenomenon
            matching_alerts = df_alerts[
                (alert_start <= dt) & 
                (alert_end >= dt) & 
                (df_alerts[phenomene_col] == phenom_id)
            ]
            
            # Get vigilance level or use default
            if len(matching_alerts) > 0:
                # If multiple alerts match, take the first one (or could aggregate)
                vigilance = matching_alerts.iloc[0][vigilance_col]
            else:
                vigilance = default_vigilance
            
            row[f'phenomene_{phenom_id}'] = vigilance
        
        pivot_data.append(row)
    
    df_pivot = pd.DataFrame(pivot_data)
    return df_pivot


# Create pivot table
print("\n" + "="*70)
print("CREATING PHENOMENA × VIGILANCE PIVOT TABLE WITH WEATHER DATA")
print("="*70)

# First, convert alert datetimes back to datetime for the pivot function
df_alerts_for_pivot = df_alerts_filtered.copy()
for col in df_alerts_for_pivot.columns:
    if 'date' in col.lower():
        df_alerts_for_pivot[col] = pd.to_datetime(df_alerts_for_pivot[col])

# Detect phenomenon and vigilance columns
phenomene_col = next((col for col in df_alerts_for_pivot.columns if 'phenomene' in col.lower()), 'phenomene_id')
vigilance_col = next((col for col in df_alerts_for_pivot.columns if 'niveau' in col.lower()), 'niveau_vigilance')

print(f"Detected phenomenon column: {phenomene_col}")
print(f"Detected vigilance column: {vigilance_col}")

# Create pivot table with weather columns (quantite_precipitations and temperature_instant)
df_pivot = create_phenomena_vigilance_pivot(
    df_hourly=df_hourly_filtered,
    df_alerts=df_alerts_for_pivot,
    datetime_col_hourly=datetime_col,
    phenomene_col=phenomene_col,
    vigilance_col=vigilance_col,
    date_start_col='date_debut_vigilance',
    date_end_col='date_fin_vigilance',
    quantite_precipitations='quantite_precipitations',
    temperature_instant='temperature_instant',
    num_phenomena=10,
    default_vigilance='Vert'
)

print(f"\n✅ Pivot Table Created:")
print(f"   Rows (unique datetimes): {len(df_pivot):,}")
print(f"   Columns: {list(df_pivot.columns)}")
print(f"   Total columns: {len(df_pivot.columns)}")

print("\n" + "="*70)
print("PIVOT TABLE STRUCTURE")
print("="*70)
print(f"Columns: datetime, quantite_precipitations, temperature_instant, phenomene_1, ..., phenomene_10")
print(f"Shape: {df_pivot.shape[0]} rows × {df_pivot.shape[1]} columns")
print("\nFirst 3 rows:")
print(df_pivot.head(3))

# ==================== ALERT COVERAGE ANALYSIS ====================
print("\n" + "="*70)
print("ALERT COVERAGE ANALYSIS")
print("="*70)

# Identify phenomenon columns (all columns except 'datetime' and weather columns)
phenom_cols = [col for col in df_pivot.columns if col.startswith('phenomene_')]

# Count rows with at least one missing alert (contains 'Vert')
rows_with_missing = (df_pivot[phenom_cols] == 'Vert').any(axis=1).sum()
pct_with_missing = (rows_with_missing / len(df_pivot)) * 100

# Count rows with ALL alerts missing (all 'Vert')
rows_all_missing = (df_pivot[phenom_cols] == 'Vert').all(axis=1).sum()
pct_all_missing = (rows_all_missing / len(df_pivot)) * 100

print(f"Total datetimes: {len(df_pivot):,}")
print(f"Datetimes with at least 1 missing alert: {rows_with_missing:,} ({pct_with_missing:.2f}%)")
print(f"Datetimes with ALL alerts missing: {rows_all_missing:,} ({pct_all_missing:.2f}%)")

# ==================== ALERT LEVEL DISTRIBUTION ====================
print("\n" + "-"*70)
print("ALERT LEVEL DISTRIBUTION")
print("-"*70)

# Flatten all vigilance values from phenomena columns
all_vigilance_values = df_pivot[phenom_cols].values.flatten()

# Count distribution by alert level
vigilance_distribution = pd.Series(all_vigilance_values).value_counts().sort_index()
total_cells = len(all_vigilance_values)

print(f"\nTotal alert cells analyzed: {total_cells:,}\n")

# Define alert level order
alert_level_order = ['Vert', 'Jaune', 'Orange', 'Rouge', 'Magenta']
for level in alert_level_order:
    if level in vigilance_distribution.index:
        count = vigilance_distribution[level]
        pct = (count / total_cells) * 100
        bar = "█" * int(pct / 2)  # Scale bar to fit terminal
        print(f"{level:12} : {count:8,} ({pct:6.2f}%) {bar}")

# Show any other levels not in standard order
for level, count in vigilance_distribution.items():
    if level not in alert_level_order:
        pct = (count / total_cells) * 100
        bar = "█" * int(pct / 2)
        print(f"{level:12} : {count:8,} ({pct:6.2f}%) {bar}")

print("\n" + "-"*70)
print("Sample of 5 random rows:")
print("-"*70)
sample_rows = df_pivot.sample(n=min(5, len(df_pivot)), random_state=42)
print(sample_rows.to_string())


Loading hourly weather data...
  ✓ Hourly weather: 33317 rows, 6 cols
Loading alert data...
  ✓ Alerts: 8378 rows, 5 cols
  ✓ Hourly weather: 33317 rows, 6 cols
Loading alert data...
  ✓ Alerts: 8378 rows, 5 cols

CREATING PHENOMENA × VIGILANCE PIVOT TABLE WITH WEATHER DATA
Detected phenomenon column: phenomene_id
Detected vigilance column: niveau_vigilance

CREATING PHENOMENA × VIGILANCE PIVOT TABLE WITH WEATHER DATA
Detected phenomenon column: phenomene_id
Detected vigilance column: niveau_vigilance

✅ Pivot Table Created:
   Rows (unique datetimes): 33,317
   Columns: ['datetime', 'quantite_precipitations', 'temperature_instant', 'phenomene_1', 'phenomene_2', 'phenomene_3', 'phenomene_4', 'phenomene_5', 'phenomene_6', 'phenomene_7', 'phenomene_8', 'phenomene_9', 'phenomene_10']
   Total columns: 13

PIVOT TABLE STRUCTURE
Columns: datetime, quantite_precipitations, temperature_instant, phenomene_1, ..., phenomene_10
Shape: 33317 rows × 13 columns

First 3 rows:
              datetime

UFuncTypeError: ufunc 'less' did not contain a loop with signature matching types (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.StrDType'>) -> None